# Import

In [1]:
!git clone -b deep-learning-models https://github.com/MohamadElnomrossie/Omdena-seniment-analysis/

fatal: destination path 'Omdena-seniment-analysis' already exists and is not an empty directory.


In [2]:
%cd Omdena-seniment-analysis/

/content/Omdena-seniment-analysis


In [3]:
# !git clone -b Tokenization https://github.com/OmdenaAI/Arabic-Chapter/

# Main

In [1]:
import os
import json

import numpy as np
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

from Sentiment import SentimentAnalysis
from utils import helper, preprocess
# from utils.config import config

# from ArabicChapter.Tokenizer.tokenizer import tokenization

### Analysis

In [2]:
import pandas as pd
data = pd.read_csv("Datasets/Final_Dataset/Dataset/train.csv")
print(data.columns)
data.head(10)

Index(['Unnamed: 0', 'Text', 'Class_camel', 'cleaned_text', 'mentions',
       'word_count', 'contain_emoji', 'demoji_text', 'URLS'],
      dtype='object')


,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1']
1,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU']
2,5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,['fatemaar7'],50,False,NaN,['https://t.co/c494qBVPx4']
3,7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"['AliFCD', 'ShamsanM', 'AlyemenNor', 'space', ...",32,False,NaN,[]
4,9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[]
5,12,إذا كانت مسألة حقوق المرأة أمرا سخيفاً جداً ، ...,negative,اذا كانت مساله حقوق المراه امرا سخيفا جدا فالل...,[],18,False,NaN,[]
6,13,@naifco كل هذه الحقوق او المزعوم انها حقوق عبا...,negative,كل هذه الحقوق او المزعوم انها حقوق عباره عن صف...,['naifco'],16,False,NaN,[]
7,14,متى تعرف ان المرأة ما عندها حقوق؟\r\n يوم يطبع...,neutral,متي تعرف ان المراه ما عندها حقوق يوم يطبعون له...,[],13,True,:backhand_index_pointing_down_medium-light_ski...,['https://t.co/7a9H0KQTqH']
8,16,RT @miss_wahaj: @RmNxb المسيار زي زواج المتعه ...,negative,المسيار زي زواج المتعه عند الشيعه نعايب عليهم ...,"['miss', 'RmNxb']",13,False,NaN,[]
9,18,RT @Gh08944301: @Eqbal_Darandari لمتى واحنا بل...,negative,لمتي واحنا بلا حقوق لانقدر نجدد جوازات سفرنا و...,"['Gh08944301', 'Eqbal']",50,False,NaN,[]


In [3]:
print(data.cleaned_text.loc[45], data.Class_camel.loc[45])
print(data.cleaned_text.loc[0], data.Class_camel.loc[0])
print(data.cleaned_text.loc[35], data.Class_camel.loc[35])

لان تبني حقوق المراه من وجهه نظر علمانيه وتحرريه يضر بمجتمعي ويضر بالنساء والرجال علي حد سواء بس هم ليش مهتمين وحاشرين خشومهم negative
حقوق المراه neutral
تضامن مع ترف العسيري اختي المسلمه حجابك حشمتك حيائك هي هويتك الحقيقيه التي كرمك بها دينك العظيم حافظي عليها فهي من تمنحك الكرامه والحريه الحقيقيه لاتستمعي لمن يحاول طمس هويتك الاسلاميه تحت دعاوي مزيفه مثل حقوق المراه وبكائيات من تسمين انفسهن ب الناشطات والحقوقيات positive


In [4]:
data.Class_camel.value_counts()

positive    57096
negative    33702
neutral     20124
Name: Class_camel, dtype: int64

### Config

In [5]:
import pickle
with open("Datasets/stopWords.pkl", 'rb') as f:
    stop_words = list(pickle.load(f))
    stop_words = list(set(stop_words + ['و','في','من','بواسطة','أ','هو','و','في','سيكون','إلى','كان','كن','هو','ال','و','ما','ء','ه','س']))

config = {
    'vocab_size':60000,
    'maxlen':256,
    'embedding_vector':200,

    'method':'lstm', #other - simpleRNN, bidRNN, 1DConv, lstm
    'stop_words':stop_words,
    'punctuations':"""'!"-#$%&'()*+,«».؛،/:؟?@[\]^_`{|}~""",

    'epochs':50,
    'optim':'SGD', # other - adamax, adadelta, SGD, Adam, RMSprop
    'learning_rate':1e-2,

    'save_model_path':'models/',
    'save_weights_path':"models/",
    'train_data_path':"Datasets/Final_Dataset/Dataset/train.csv",
    'val_data_path':"Datasets/Final_Dataset/Dataset/val.csv",
    'test_data_path':"Datasets/Final_Dataset/Dataset/test.csv",
}


### Fine Tuning

In [8]:
train_data = pd.read_csv(config['train_data_path'])
val_data = pd.read_csv(config['val_data_path'])
test_data = pd.read_csv(config['test_data_path'])

train_data = train_data.dropna().reset_index(drop=True)
val_data = val_data.dropna().reset_index(drop=True)
test_data = test_data.dropna().reset_index(drop=True)

train_text, train_label = train_data['cleaned_text'].values, train_data['Class_camel'].values
val_text, val_label = val_data['cleaned_text'].values, val_data['Class_camel'].values
test_text, test_label = test_data['cleaned_text'].values.copy(), test_data['Class_camel'].values.copy()


In [9]:
#Default Tokenization
sentiment = SentimentAnalysis(preprocess.tokenizer, vocab_size=config['vocab_size'], maxlen=config['maxlen'], embedding_vector=config['embedding_vector'], method=config['method'],)

train_text = sentiment.tokenize(train_text, punctuations=config['punctuations'], stop_words=config['stop_words'])
train_text, train_label, unique_words, word_dict = sentiment.vectorize(train_text, train_label, return_label=True)

val_text = sentiment.tokenize(val_text, punctuations=config['punctuations'], stop_words=config['stop_words'])
val_text, val_label, _, _ = sentiment.vectorize(val_text, val_label, return_label=True)

test_text = sentiment.tokenize(test_text, punctuations=config['punctuations'], stop_words=config['stop_words'])
test_text, test_label, _, _ = sentiment.vectorize(test_text, test_label, return_label=True)


In [10]:
train_data.cleaned_text.iloc[0], train_data.Class_camel.iloc[0] 

(['حقوق', 'المراه'], 'neutral')

In [12]:
train_text[0], train_label[0]

(array([49698,  1937,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [13]:
model = sentiment.fit(train_text,
                    train_label,
                    validation_data=(val_text, val_label),
                    epochs=config['epochs'],
                    method=config['method'])
sentiment.evaluate(test_text, test_label, model, batch_size=32)

(29029, 256) (7267, 256) (29029, 3) (7267, 3)
Epoch 1/50
908/908 [==============================] - 49s 45ms/step - loss: 0.9982 - accuracy: 0.4924 - precision: 0.4985 - recall: 0.2021 - auc: 0.6787 - val_loss: 0.9777 - val_accuracy: 0.4900 - val_precision: 0.5120 - val_recall: 0.4728 - val_auc: 0.7101

Epoch 00001: val_loss improved from inf to 0.97767, saving model to models\lstm_model.h5
Epoch 2/50
908/908 [==============================] - 40s 44ms/step - loss: 0.9566 - accuracy: 0.5003 - precision: 0.5618 - recall: 0.2435 - auc: 0.7124 - val_loss: 0.8954 - val_accuracy: 0.5863 - val_precision: 0.7962 - val_recall: 0.2393 - val_auc: 0.7754

Epoch 00002: val_loss improved from 0.97767 to 0.89544, saving model to models\lstm_model.h5
Epoch 3/50
908/908 [==============================] - 40s 44ms/step - loss: 0.8243 - accuracy: 0.6085 - precision: 0.7006 - recall: 0.4291 - auc: 0.8064 - val_loss: 0.7820 - val_accuracy: 0.6287 - val_precision: 0.6748 - val_recall: 0.5572 - val_auc: 0.8

### Validation

In [14]:
print(f"\nText : {test_data['cleaned_text'].iloc[45]} Label : {test_data['Class_camel'].iloc[45]}")
sentiment.predict_([test_data['cleaned_text'].iloc[45]], model, batch_size=32, print_=True)

print(f"Text : {test_data['cleaned_text'].iloc[0]} Label : {test_data['Class_camel'].iloc[0]}")
sentiment.predict_([test_data['cleaned_text'].iloc[0]], model, batch_size=32, print_=True)

print(f"Text : {test_data['cleaned_text'].iloc[35]} Label : {test_data['Class_camel'].iloc[35]}")
sentiment.predict_([test_data['cleaned_text'].iloc[35]], model, batch_size=32, print_=True)


Text : سعوديات نطلب اسقاط الولايه سعوديات نطلب اسقاط الولايه هلو بابا سلمان طولتوا مو قال محمد بن سلمان ان الولايه راح تسقط قريب متي هالقريب تري راح عمرنا واحنا ننتظر اسقوطها بليز واعتقونا Label : negative



--------------------
Neutral 0.5938876867294312
--------------------
Text : مبدائيا انا مع حقوق المراه وضد حقوق الرجل Label : neutral



--------------------
Positive 0.5096917152404785
--------------------
Text : تضامن مع ترف العسيري فئه اسقاط الولايه القذره حثاله المجتمع الذين سمعوا عن الحريه فنادوا بخلع الحجاب والملابس ويعتبرونها حريهالشخص الحجاب عباده Label : negative



--------------------
Negative 0.9999344348907471
--------------------


### Inference

In [16]:
import pandas as pd
import tensorflow as tf
from Sentiment import SentimentAnalysis
from utils import helper, preprocess

data = pd.read_csv("test.csv")
data.head(10)

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,neutral,حقوق المراه التي تضمنها لها وزاره العدل,['MojKsa'],7,False,NaN,['https://t.co/QUGzWwubFk']
1,4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,neutral,ولي امر الزوجه او ولي الزوجه او ولي المراه من ...,['abm112211'],47,False,NaN,['https://t.co/rkqxvm5qY7']
2,8,مبدائيا انا مع حقوق المرأة وضد حقوق الرجل 😁,neutral,مبدائيا انا مع حقوق المراه وضد حقوق الرجل,[],8,True,:beaming_face_with_smiling_eyes:,[]
3,19,RT @Stranger23_: @Oh__28 @LifeRights19 عشان كذ...,negative,عشان كذا انا عندي تحفظ للي يشوف الغاء الولايه ...,"['Stranger23', 'Oh', 'LifeRights19']",49,True,:downcast_face_with_sweat:,[]
4,34,@Eqbal_Darandari @g11_we المفروض تتم الموافقة ...,neutral,المفروض تتم الموافقه وبلاجماع مافي ابدا توافق ...,"['Eqbal', 'g11']",25,False,NaN,[]
5,36,بندخل 2019 وأنتم للحين عايشين بالقرون الوسطى ت...,negative,بندخل وانتم للحين عايشين بالقرون الوسطي تطاردو...,[],32,False,NaN,[]
6,37,@mosam20_mo طبعا حقوق المرأة المتنازل عنها \r\...,neutral,طبعا حقوق المراه المتنازل عنها النفقهو السكن و...,['mosam20'],22,False,NaN,[]
7,43,RT @the0truth: ستظل مسألة حقوق المرأة مسألة شا...,negative,ستظل مساله حقوق المراه مساله شائكه خاصه بدول ا...,['the0truth'],28,False,NaN,[]
8,44,@Badria_AlShihi ومن قال ان الانوثة تنزع من الم...,negative,ومن قال ان الانوثه تنزع من المراه حين تطالب بح...,['Badria'],43,False,NaN,[]
9,46,@BadriyaRM @___IHAVENOIDEA المرأه بالاسلام تلع...,negative,المراه بالاسلام تلعن اذا رفضت زوجها بالفراش وا...,['BadriyaRM'],42,False,NaN,[]


In [17]:
data = data.dropna().reset_index(drop=True)
text = data["Text"].values

In [18]:
text[0:5]

array(['مبدائيا انا مع حقوق المرأة وضد حقوق الرجل 😁',
       'RT @Stranger23_: @Oh__28 @LifeRights19 عشان كذا أنا عندي تحفظ للي يشوف الغاء الولاية عندنا هو الحل النهائي! نعم حو حق من حقوق المرأة وبيحل جزء من المشكلة، لكن حتى لو الغيت الولاية سطوة المجتمع والعائلة ستظل موجودة بل حتى جرائم الشرف منتشرة في دول ماعندها ولاية اكثر بكثير من اللي عندنا! (نظرة تشاؤمية 😓)',
       '@jWYVXK0PCumva4b @Just_94376 @umdorah4 هذا حق من حقوق المرأة\r\n مثل ما الرجل يريد وحده زينه واخلاق ومتوظفه او لا .. ويتشرط هي بعد.. الحياة الزوجية ليست للرجل فقط انما للاثنيين ولازم تكون مقتنعه فيه ☺\r\n \r\n تقبلوا احترامي 🌹',
       'جمعيات حقوق المرآه والمنظمات والمؤسسات و(strong women)كل دا مالوش اى لازمه بالنسبه لنا,احنا بس بنفكركم بقيمتنا الاانتوا نسيتوها .علشان فى الاول والاخر البنت هتبقا محتاجه ابوها يحميها مش موجود يبقا اخوها مش موجود يبقا جوزها مش موجود يبقا ابنها ومستحيل هتعيش لوحدها .❤#اسآل_جوليان',
       '@Free88_ حقوق المرأة أنها تكون مع رجل اخر ؟🙃🤢'], dtype=object)

In [19]:
model = tf.keras.models.load_model("models/lstm_model.h5")
preds = sentiment.predict_(text, model, batch_size=32, print_=False)
data["prediction"] = preds

In [20]:
data.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS,prediction
0,8,"[مبدائيا, انا, مع, حقوق, المرأة, وضد, حقوق, ال...",neutral,مبدائيا انا مع حقوق المراه وضد حقوق الرجل,[],8,True,:beaming_face_with_smiling_eyes:,[],Negative
1,19,"[RT, Oh, عشان, كذا, أنا, عندي, تحفظ, للي, يشوف...",negative,عشان كذا انا عندي تحفظ للي يشوف الغاء الولايه ...,"['Stranger23', 'Oh', 'LifeRights19']",49,True,:downcast_face_with_sweat:,[],Negative
2,48,"[Just, هذا, حق, حقوق, المرأة, مثل, الرجل, يريد...",positive,هذا حق من حقوق المراه مثل ما الرجل يريد وحده ز...,"['jWYVXK0PCumva4b', 'Just', 'umdorah4']",31,True,:smiling_face::rose:,[],Negative
3,55,"[جمعيات, حقوق, المرآه, والمنظمات, والمؤسسات, s...",negative,جمعيات حقوق المراه والمنظمات والمؤسسات وكل دا ...,[],43,True,:red_heart:,[],Negative
4,94,"[حقوق, المرأة, أنها, تكون, مع, رجل, اخر, 🙃🤢]",neutral,حقوق المراه انها تكون مع رجل اخر,['Free88'],7,True,:upside-down_face::nauseated_face:,[],Neutral
